In [29]:
%pylab nbagg
from tvb.simulator.lab import *
import pandas as pd
import h5py

from tvb.simulator.lab import *
import h5py
import os
import matplotlib.pyplot as plt

import scipy 
import scipy.fftpack
import seaborn as sns

import scipy 
import scipy.fftpack
from scipy.fft import fft, ifft
from scipy import signal
from scipy.integrate import simps
from fooof import FOOOF

Populating the interactive namespace from numpy and matplotlib


C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f', 'indices']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#depressed brain 
Q_health = np.repeat(1, 76)
oscilator = models.WilsonCowan(Q = Q_health)

In [4]:
white_matter = connectivity.Connectivity.from_file()
white_matter.speed = numpy.array([3.0])

WARNING  File 'hemispheres' not found in ZIP.


In [5]:
white_matter_coupling = coupling.Linear(a=numpy.array([0.0039]))

In [16]:
#Initialise some Monitors with period in physical time
mon_raw = monitors.Raw(period = 0.9765625)
mon_tavg = monitors.TemporalAverage(period = 0.9765625)

#Bundle them
what_to_watch = (mon_raw, mon_tavg)

In [17]:
#important variables of dataset
sample_period = 0.0009765625 #seconds         
sf = 1/sample_period #sampling frequency 
sample_rate = 1024

#sns.set(font_scale=1.2)
dt = 0.01

In [18]:
os.chdir("C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Wilson Cowan/DATA")
#import connectivity file to know which region corresponds to which index in the matrices
filename = "Connectivity.h5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    regions_list = list(f['region_labels'])

regions = np.array(regions_list)


Keys: <KeysViewHDF5 ['areas', 'centres', 'cortical', 'hemispheres', 'orientations', 'region_labels', 'tract_lengths', 'weights']>


In [27]:
def powercalc(data, low, high, sf, win):
    freqs, psd = signal.welch(data, sf, nperseg=win)
    freq_res = freqs[1] - freqs[0]
    total_power = simps(psd, dx=freq_res)
    idx= np.logical_and(freqs >= low, freqs <= high)
    power = simps(psd[idx], dx=freq_res)
    return power

In [42]:
#loop  start om 9u56
mydir = "C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Datasets/All_freqs"

c1 = np.repeat("", 1)
c2 = np.repeat("",1)

ROI_base = numpy.zeros(shape=(1,2))
ROI_ar = np.column_stack([c1, c2, ROI_base])
ROI_tot= np.column_stack([c1, c2, ROI_base])

power_arr = numpy.zeros(shape = (1,5))
power_arr_right =numpy.zeros(shape = (1,5))
power_arr_left = numpy.zeros(shape = (1,5))
tot_power_arr = numpy.zeros(shape = (1,5))
tot_power_arr_right =numpy.zeros(shape = (1,5))
tot_power_arr_left = numpy.zeros(shape = (1,5))

varkeep = {}
varhigh = {}
varall = {}
TSkeep = {}
leftDLPFC_keep = {}
rightDLPFC_keep = {}

win = 4 * sf
sf = 1024

indices = [7,  13, 15, 16, 17,18, 19, 20, 21, 30, 31, 33,35, 36, 45, 51, 53, 54, 55, 56, 57, 58, 59, 68, 69, 71 , 73,74]
NS_arr = np.arange(0,26, 0.25)

counter = 0
loop = 1

nsubs = 101

for i in range(nsubs): 
    if i == 0:
        pass 
    else:
        value = 10**-6 #noise dispertion
        NS = NS_arr[i] #changing noise seed
        print(loop)
        
        #create simulator 
        sim = simulator.Simulator(model = oscilator, connectivity = white_matter,
                          coupling = white_matter_coupling, 
                          integrator = integrators.EulerStochastic(dt=0.01220703125, noise=noise.Additive(noise_seed = int(NS), nsig=numpy.array([value]))),
                        monitors =  what_to_watch)

        sim.configure()

        print("check1")
        
        #Perform the simulation
        tavg_data = []
        tavg_time = []

        for raw, tavg in sim(simulation_length=2500):
    
            if not tavg is None:
                tavg_time.append(tavg[0])
                tavg_data.append(tavg[1])
    
        #Make the lists numpy.arrays for easier use.
        RAW = numpy.array(tavg_data)
        RAW_real = RAW[:, 0, :, 0]
        
        if loop == 1:
            sf = len(RAW_real)/2.5 #calculate sampling frequency
            print(sf)
            print(len(tavg_time))
            print(shape(RAW_real))
        
        print("check2")
        #delete the first 500ms
        list_deleterows = [*range(0,500,1)]
        RAW_real = np.delete(RAW_real, list_deleterows, 0)
        tavg_time =  np.delete(tavg_time, list_deleterows, 0)
        
        #save average time series (across regions)
        AV = RAW_real
        AV_ar = AV.mean(axis = 1) 
        TSkeep["data" + str(i)] = AV_ar
        
        #save time series of left DLPFC (for alpha asymmetry later)
        leftDLPFC = RAW_real[:,56]
        leftDLPFC_keep["data" + str(i)] = leftDLPFC

        #save time series of right DLPFC (for alpha asymmetry later)
        rightDLPFC = RAW_real[:,18]
        rightDLPFC_keep["data"+str(i)] = rightDLPFC
        
        #store PSDs
        #psd for all brain 
        for x in range(5):            
            low, high = 8+x, 9+x
            if x == 5:
                low, high = 8, 12
            
            power_arr[0][x] = powercalc(AV_ar, low, high, sf, win)
            power_arr_left[0][x] = powercalc(leftDLPFC, low, high, sf, win)
            power_arr_right[0][x] = powercalc(rightDLPFC, low, high, sf, win)
            
        if loop == 1:
            tot_power_arr = numpy.copy(power_arr)
            tot_power_arr_left = numpy.copy(power_arr_left)
            tot_power_arr_right = numpy.copy(power_arr_right)
        else:
            tuplePower = (tot_power_arr, power_arr)
            tot_power_arr = np.vstack(tuplePower)
                
            tuplePower_left = (tot_power_arr_left, power_arr_left)
            tot_power_arr_left = np.vstack(tuplePower_left)
                
            tuplePower_right = (tot_power_arr_right, power_arr_right)
            tot_power_arr_right = np.vstack(tuplePower_right)
                
            
        loop = loop +1
    
#compute complete array containing all time series of all 100simulations   
namelist4 = []
for i in range(nsubs): #range(101):
    if i == 0:
        pass
    else:
        name4 = TSkeep["data" + str(i)]
        namelist4.append(name4)

nametup4 = tuple(namelist4)
array_TS = np.column_stack(nametup4)

print(shape(array_TS))

#make dataset containing all 100 right DLPFC time series + average across them
namelist5 = []
for i in range(nsubs): #range(101):
    if i == 0:
        pass
    else:
        name5 = rightDLPFC_keep["data"+str(i)] 
        namelist5.append(name5)

nametup5 = tuple(namelist5)
array_rDLPFC = np.column_stack(nametup5)


#make dataset containing all 100 left DLPFC time series 
namelist6 = []
for i in range(nsubs): #range(101):
    if i == 0:
        pass
    else:
        name6 = leftDLPFC_keep["data"+str(i)] 
        namelist6.append(name6)

nametup6 = tuple(namelist6)
array_lDLPFC = np.column_stack(nametup6)

file_name = mydir + "/power_alpha_overall.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=tot_power_arr)
h5f.close()

file_name = mydir + "/power_alpha_rDLPFC.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=tot_power_arr_right)
h5f.close()

file_name = mydir + "/power_alpha_lDLPFC.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=tot_power_arr_left)
h5f.close()

file_name = mydir + "/Health_rightDLPFC.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=array_rDLPFC)
h5f.close()

file_name = mydir + "/Health_leftDLPFC.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=array_lDLPFC)
h5f.close()
        
file_name = mydir + "/Time_Health.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('time', data=tavg_time)
h5f.close()

file_name = mydir + "/TimeSeries_Health.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('time', data=array_TS )
h5f.close()

1
check1
1024.0
2560
(2560, 76)
check2
2
check1


C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\cel

check2
3
check1


C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\cel

check2
4
check1


...

C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\cel

check2
100
check1


C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\cel

check2
(2060, 100)


C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\celie\OneDrive\Documenten\TVB_Windows_2.2\TVB_Distribution\tvb_data\Lib\site-packages\scipy\signal\spectral.py:1963: UserWarning: nperseg = 4096 is greater than input length  = 2060, using nperseg = 2060
  .format(nperseg, input_length))
C:\Users\cel

In [25]:
power_arr = numpy.zeros(shape = (1,5))
print(power_arr)

[[0. 0. 0. 0. 0.]]


In [37]:
tot_power_arr

array([[8.81101998e-09, 4.92501532e-10, 8.06132517e-10, 3.61543834e-09,
        8.68855379e-09],
       [8.81101998e-09, 4.92501532e-10, 8.06132517e-10, 3.61543834e-09,
        8.68855379e-09],
       [8.81101998e-09, 4.92501532e-10, 8.06132517e-10, 3.61543834e-09,
        8.68855379e-09],
       [8.42003317e-10, 7.18399182e-10, 1.91118639e-09, 1.08688654e-09,
        2.29737676e-09]])

In [ ]:
array = np.hstack(nametup)

In [ ]:
print(shape(array))

In [ ]:
print(array)

In [39]:
file_name = mydir + "/power_alpha_overall.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=tot_power_arr)
h5f.close()

file_name = mydir + "/power_alpha_rDLPFC.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=tot_power_arr_right)
h5f.close()

file_name = mydir + "/power_alpha_lDLPFC.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=tot_power_arr_left)
h5f.close()

In [41]:
print(shape(tot_power_arr))

(14, 5)
